In [1]:
import os
import math
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer, Normalizer, StandardScaler
from sklearn.externals import joblib

In [2]:
os.getcwd()

'/Users/dominikpeter/Homework/mining/Uebung2'

In [3]:
x = [x for x in os.walk('/Users/dominikpeter/Homework/mining/HMP_Dataset')]

In [4]:
subfolder = x[0][1]
subfolder[0:5]

['Brush_teeth',
 'Climb_stairs',
 'Climb_stairs_MODEL',
 'Comb_hair',
 'Descend_stairs']

In [5]:
folders = []
[folders.append('/Users/dominikpeter/Homework/mining/HMP_Dataset' + '/' + word) for word in subfolder]
folders[0:5]

['/Users/dominikpeter/Homework/mining/HMP_Dataset/Brush_teeth',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Climb_stairs',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Climb_stairs_MODEL',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Comb_hair',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Descend_stairs']

In [6]:
files = []
for folder in folders:
    [files.append(folder + '/' + f) for f in os.listdir(folder) if f.endswith(".txt")]
files[0:5]

['/Users/dominikpeter/Homework/mining/HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt']

In [7]:
df = []
for file in files:
    new_df = pd.read_csv(file, sep = ' ', header=None, names = ['x', 'y', 'z'])
    new_df['label'] = re.search('-\w+-', file[::-1]).group(0)[::-1].replace('-', '')
    df.append(new_df)
df = pd.concat(df, axis=0)
df.head()

,x,y,z,label
0,22,49,35,brush_teeth
1,22,49,35,brush_teeth
2,22,52,35,brush_teeth
3,22,52,35,brush_teeth
4,21,52,34,brush_teeth


In [8]:
df, final = train_test_split(df, test_size=0.05, random_state=42)
final.copy(deep=True)
print(df.shape)
print(final.shape)

(455324, 4)
(23965, 4)


In [9]:
df = df.join(df.groupby('label')[['x', 'y','z']].mean(), on='label', rsuffix='.mean')
df.head()

,x,y,z,label,x.mean,y.mean,z.mean
50,8,35,32,walk,13.731728,37.926308,35.492101
1844,46,30,47,use_telephone,42.540178,34.308367,46.382920
219,42,30,50,drink_glass,35.961100,38.180452,49.059692
258,11,37,38,getup_bed,22.617680,35.520320,42.387810
105,20,39,48,standup_chair,19.495057,38.471185,40.862149


In [10]:
df['diff.x'] = abs(df['x'] - df['x.mean'])
df['diff.y'] = abs(df['y'] - df['y.mean'])
df['diff.z'] = abs(df['z'] - df['z.mean'])
df['max.y'] = max(df['y'])

In [11]:
df.head()

,x,y,z,label,x.mean,y.mean,z.mean,diff.x,diff.y,diff.z,max.y
50,8,35,32,walk,13.731728,37.926308,35.492101,5.731728,2.926308,3.492101,63
1844,46,30,47,use_telephone,42.540178,34.308367,46.382920,3.459822,4.308367,0.617080,63
219,42,30,50,drink_glass,35.961100,38.180452,49.059692,6.038900,8.180452,0.940308,63
258,11,37,38,getup_bed,22.617680,35.520320,42.387810,11.617680,1.479680,4.387810,63
105,20,39,48,standup_chair,19.495057,38.471185,40.862149,0.504943,0.528815,7.137851,63


In [12]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['label'])

In [13]:
X = df.iloc[:, np.delete(np.arange(10),3)]
X.head()
#X = df.iloc[:, [0,1,2]]

,x,y,z,x.mean,y.mean,z.mean,diff.x,diff.y,diff.z
50,8,35,32,13.731728,37.926308,35.492101,5.731728,2.926308,3.492101
1844,46,30,47,42.540178,34.308367,46.382920,3.459822,4.308367,0.617080
219,42,30,50,35.961100,38.180452,49.059692,6.038900,8.180452,0.940308
258,11,37,38,22.617680,35.520320,42.387810,11.617680,1.479680,4.387810
105,20,39,48,19.495057,38.471185,40.862149,0.504943,0.528815,7.137851


In [14]:
normalizer = Normalizer().fit(X)
X = normalizer.transform(X)
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [16]:
pca = PCA(n_components=3)
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)

In [17]:
print(X.shape, y.shape)

(455324, 9) (455324, 20)


In [18]:
model1 = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, n_jobs=4), n_jobs=4)
model1.fit(X_train, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=4, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          n_jobs=4)

In [19]:
#model2 = OneVsRestClassifier(GaussianNB(), n_jobs=4)
#model2.fit(X_train, y_train)

In [20]:
model3 = OneVsRestClassifier(LogisticRegression(), n_jobs=4)
model3.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=4)

In [ ]:
model4 = OneVsRestClassifier(AdaBoostClassifier(n_estimators=100), n_jobs=4)
model4.fit(X_train, y_train)

OneVsRestClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None),
          n_jobs=4)

In [ ]:
eclf = OneVsRestClassifier(
    VotingClassifier(estimators=[('raf', model1)
                                 #,('nab', model2)
                                 ,('lor', model3)
                                 ,('ada', model4)
                                ], voting='soft', weights=[3,1,2]), n_jobs=3)

# Trainieren des VotingClassifiers
eclf = eclf.fit(X_train, y_train)

In [ ]:
y_score = eclf.predict(X_test)
y_true = y_test

In [ ]:
roc_auc_score(y_true, y_score, average='macro', sample_weight=None)

In [ ]:
accuracy_score(y_true, y_score, normalize=True, sample_weight=None)

In [ ]:
# test with unseen data
final = final.join(final.groupby('label')[['x', 'y','z']].mean(), on='label', rsuffix='.mean')

In [ ]:
#seperat calculate mean and stats
final['diff.x'] = abs(final['x'] - final['x.mean'])
final['diff.y'] = abs(final['y'] - final['y.mean'])
final['diff.z'] = abs(final['z'] - final['z.mean'])
final['max.y'] = max(final['y'])

mlb = MultiLabelBinarizer()
y_final = mlb.fit_transform(final['label'])
X_final = final.iloc[:, np.delete(np.arange(10),3)]
normalizer = Normalizer().fit(X_final)
X_final = normalizer.transform(X_final)
scaler = StandardScaler().fit(X_final)
X_final = scaler.transform(X_final)

In [ ]:
y_final_predict = eclf.predict(X_final)
y_final_true = y_final

print(roc_auc_score(y_final_true, y_final_predict, average='macro',sample_weight=None))
print(accuracy_score(y_final_true, y_final_predict, normalize=True, sample_weight=None))
#not bad

In [ ]:
joblib.dump(eclf, 'HMP-model.pkl')

In [ ]:
# test loading
eclf = joblib.load('HMP-model.pkl') 

In [ ]:
y_final_predict = eclf.predict(X_final)
y_final_true = y_final

print(roc_auc_score(y_final_true, y_final_predict, average='macro',sample_weight=None))
#not bad
print(classification_report(y_final_predict, y_final_true))